In [1]:
import sys
# set python syspath to point out location of our self-writing module
sys.path.append("/home/ponshane/work_dir/CLTM/src/codebase/")

from LanguageEmbedding import LanguageEmbedding

# UM-Corpus

In [ ]:
counts = []

In [ ]:
um_cn = "/home/ponshane/work_dir/CLTM-Experiments/Data/UM-Corpus/800K-sampled-docs/800K_Chinese_UM_Corpus.txt"
with open(um_cn, "r") as handler:
    cn_dict = {}
    for line in handler:
        temp = line.strip("\n").split(" ")
        counts.append(len(temp))
        for word in temp:
            if word not in cn_dict.keys():
                cn_dict[word] = True

In [ ]:
print(len(cn_dict), sum(counts)/len(counts))

In [ ]:
um_en = "/home/ponshane/work_dir/CLTM-Experiments/Data/UM-Corpus/800K-sampled-docs/800K_English_UM_Corpus.txt"
with open(um_en, "r") as handler:
    en_dict = {}
    for line in handler:
        temp = line.strip("\n").split(" ")
        counts.append(len(temp))
        for word in temp:
            if word not in en_dict.keys():
                en_dict[word] = True

In [ ]:
print(len(en_dict), sum(counts)/len(counts))

In [ ]:
concatenate_word_vectors = LanguageEmbedding.read_from_KeyedVectors("/home/ponshane/work_dir/CLTM-Experiments/Data/UM-Corpus/word-vectors/2018-09-27-ponshane-um-concatenate-wordvec-mikolov-100d.vec")

In [12]:
dict_path = "/home/ponshane/work_dir/CLTM-Experiments/Data/UM-Corpus/50K-sampled-docs/um-corpus-news-top-translation.csv"
valid_pairs = {}
with open(dict_path, "r") as handler:
    for line in handler:
        temp = line.strip("\n").split(",")
        if temp[0] in concatenate_word_vectors.index2word and temp[1] in concatenate_word_vectors.index2word:
            valid_pairs[temp[1]] = temp[0]

In [ ]:
len(valid_pairs)

In [ ]:
LanguageEmbedding.evaluate_translation_precision(concatenate_word_vectors, valid_pairs, print_match_pair=False)

# MLDoc

In [ ]:
counts = []

In [ ]:
um_cn = "/home/ponshane/work_dir/CLTM-Experiments/Data/MLDoc/MLDoc-Chinese.txt"
with open(um_cn, "r") as handler:
    cn_dict = {}
    for line in handler:
        temp = line.strip("\n").split(" ")
        counts.append(len(temp))
        for word in temp:
            if word not in cn_dict.keys():
                cn_dict[word] = True

In [ ]:
print(len(cn_dict), sum(counts)/len(counts))

In [ ]:
um_en = "/home/ponshane/work_dir/CLTM-Experiments/Data/MLDoc/MLDoc-English.txt"
with open(um_en, "r") as handler:
    en_dict = {}
    for line in handler:
        temp = line.strip("\n").split(" ")
        counts.append(len(temp))
        for word in temp:
            if word not in en_dict.keys():
                en_dict[word] = True

In [ ]:
print(len(en_dict), sum(counts)/len(counts))

In [29]:
concatenate_word_vectors = LanguageEmbedding.read_from_KeyedVectors("/home/ponshane/work_dir/CLTM-Experiments/Data/MLDoc/word-vectors/Chinese_English_wordvectors.vec")

In [30]:
dict_path = "/home/ponshane/Downloads/zh-en.txt"
valid_pairs = {}
with open(dict_path, "r") as handler:
    for line in handler:
        temp = line.strip("\n").split(" ")
        if temp[0] in concatenate_word_vectors.index2word and temp[1] in concatenate_word_vectors.index2word:
            valid_pairs[temp[0]] = temp[1]

In [31]:
len(valid_pairs)

607

In [32]:
LanguageEmbedding.evaluate_translation_precision(concatenate_word_vectors, valid_pairs, print_match_pair=False)

Precision @1: 0.053, Precision @5: 0.129, Precision @10: 0.198, Precision @30 0.359


In [ ]:
# evaluate for hand-craft dictionary
concatenate_word_vectors = LanguageEmbedding.read_from_KeyedVectors("/home/ponshane/work_dir/CLTM-Experiments/Data/MLDoc/word-vectors/Procrustes_Hand_craft_Chinese_English_wordvectors.vec")

dict_path = "/home/ponshane/work_dir/CLTM-Experiments/Data/MLDoc/hand-craft-control-dictionary.txt"
valid_pairs = {}
with open(dict_path, "r") as handler:
    for line in handler:
        temp = line.strip("\n").split(",")
        if temp[0] in concatenate_word_vectors.index2word and temp[1] in concatenate_word_vectors.index2word:
            # notice the order!
            # {chinese: english}
            valid_pairs[temp[1]] = temp[0]

print(len(valid_pairs))

LanguageEmbedding.evaluate_translation_precision(concatenate_word_vectors, valid_pairs, print_match_pair=False)

# Hand_craft_Chinese_English_wordvectors.vec
# Precision @1: 0.063, Precision @5: 0.217, Precision @10: 0.305, Precision @30 0.496

# Procrustes_Hand_craft_Chinese_English_wordvectors.vec
# Precision @1: 0.016, Precision @5: 0.083, Precision @10: 0.128, Precision @30 0.258

In [ ]:
import configparser
import pymongo
from pymongo import MongoClient
from datetime import datetime
import pickle
import re
import gensim

import sys
# set python syspath to point out location of our self-writing module
sys.path.append("/home/ponshane/work_dir/CLTM/src/codebase/")

from helper import *

### init and read config
config = configparser.ConfigParser()
config.read('../config.ini')

MongoDB = config["MLDoc"]["Database"]
MongoUser = config["MLDoc"]["User"]
MongoPW = config["MLDoc"]["PW"]

###連接MONGO
uri = "mongodb://" + MongoUser + ":" + MongoPW + "@140.117.69.70:30241/" + MongoDB + "?authMechanism=SCRAM-SHA-1"

client = MongoClient(uri)
db = client.MLDoc

# For Chinese

In [ ]:
### 檢查單字是否都是 英文字/數字 http://hzy3774.iteye.com/blog/2359032
def judge_pure_english(keyword):  
    return all(ord(c) < 128 for c in keyword)

def extract_selected_pos_word(sentence):
    # 挑取 #N, #V 詞性開頭的字
    indices = [i for i, s in enumerate(sentence) if "#N" in s or "#V" in s]
    select_words = [sentence[index] for index in indices]
    
    # 清除詞性標籤
    words = [re.search('(.*)#', word).group(1) for word in select_words]
    
    # 過濾單詞
    words = [word for word in words if len(word) >= 2]
    
    # 過濾英文字
    words = [word for word in words if judge_pure_english(word) == False]
    return(words)

In [ ]:
start_time = datetime.now()

target_collection = db.Chinese
query_documents = target_collection.find({"chi_nlp_process": {"$exists": True}}, no_cursor_timeout=True)

docs = list()
index = 0

for each_document in query_documents:
    sentence = []
    for each_sentence in each_document["chi_result"]["pos"]:
        sentence += extract_selected_pos_word(each_sentence)
    docs.append(sentence)
    
    index += 1
    if(index % 1000 ==0):
        print("Already process %d documents" % index)

query_documents.close()

time_elapsed = datetime.now() - start_time 

print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
# Time elapsed (hh:mm:ss.ms) 0:00:18.737113

In [ ]:
temp = 0
for doc in docs:
    temp += len(doc)
    
temp/len(docs)

In [ ]:
start_time = datetime.now()

target_collection = db.English
mongo_docs = target_collection.find({"nlp_process": {"$exists": True}}, no_cursor_timeout=True)

index = 0
docs = []
id_mongo_dictionary = {}

for doc in mongo_docs:
    sentence = []
    for each_sentence in doc["nested_token_list"]:
        tokens_from_each_sentence = project_function_for_every_document(each_sentence, want_stop=False,
                                                                        want_alpha=True, want_lemma=True,
                                                                        accept_pos = ["NOUN", "VERB"],
                                                                        use_entity=False)
        sentence += tokens_from_each_sentence
        
    docs.append(sentence)

    index += 1
    if(index % 5000 ==0):
        print("Already process %d documents" % index)
        
mongo_docs.close()

time_elapsed = datetime.now() - start_time

print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
# Time elapsed (hh:mm:ss.ms) 0:35:12.122783

In [ ]:
temp = 0
for doc in docs:
    temp += len(doc)
    
temp/len(docs)

In [ ]:
with open("/home/ponshane/work_dir/CLTM-Experiments/Data/UM-Corpus/800K-sampled-docs/selected800KDos.txt", "r") as handler:
    docs = []
    for line in handler:
        docs.append(len(line.strip("\n").split(" ")))

In [ ]:
sum(docs)/len(docs)